In [1]:
include("lib/all.jl");

In [2]:
particles = make_particles(2^20);
t = Tree(particles);

In [12]:
@time group!(t, 8);

  0.110248 seconds (4 allocations: 160 bytes)


In [13]:
@time inform!(t)

  0.023554 seconds (4 allocations: 160 bytes)


In [14]:
test_in_cell_mass(t)

In [15]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [16]:
@time get_all_accel!(t, 0.5, 0.0001, ax,ay,az, 8)

 11.278110 seconds (4 allocations: 160 bytes)


In [ ]:
@time get_all_accel_rel!(t, 0.025, 0.0001, ax,ay,az, 8)

In [19]:
perf(particles, ax, 1000, 0.0001)

112.67979626285509